# 1. Setup lbraries

In [ ]:
import librosa
import os
import numpy as np
import glob
from glob import iglob
import matplotlib.pyplot as plt
import librosa.display
import random

# 2. Define augmentation functions and path to folders

In [ ]:
n_songs = 100
rootdir_glob = "../data/genres/**"
save_path = "../data/genres_aug/"
count = 0
def manipulate(data, noise_factor):
    noise = np.random.randn(len(data))
    augmented_data = data + noise_factor * noise
    # Cast back to same data type
    augmented_data = augmented_data.astype(type(data[0]))
    return augmented_data

def shift(data, sampling_rate, shift_max, shift_direction):
    shift = sampling_rate*shift_max
    shift_data = [0]*shift
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data=np.concatenate((augmented_data, shift_data), axis=0)
    else:
        augmented_data = np.concatenate((shift_data,augmented_data),axis=0)
    return augmented_data

def pitch_shift(data, sampling_rate, pitch_factor):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

def speed_change(data, speed_factor,sampleRate):
    data = librosa.effects.time_stretch(data, speed_factor)
    data = shift(data,sampleRate,5,'both') #Shifting the song to make it 30 secs long.
    return data

# 3. Augment data

Augmentation chosen:
    - add noise
    - add more noise
    - change pitch
    - speed up
    - speed down
    
After augmenting, the dataset will be 6 times larger

In [ ]:
for filename in glob.iglob(rootdir_glob, recursive=True):
    if os.path.isfile(filename):
        x, sampleRate = librosa.load(filename, duration=30)
        save_folder = filename.split('\\')[1] #Get the genres folder
        name = filename.split('\\')[2]
        
        # add noise
        noise_factor = np.random.normal(0,2)/100
        augment_data = manipulate(x,noise_factor)
        save_fileName = save_path  + save_folder + "/" + "aug1_" + name
        librosa.output.write_wav(save_fileName , augment_data, sampleRate)
        
        # add MORE noise!
        noise_factor = np.random.normal(2,4)/100
        augment_data = manipulate(x,noise_factor)
        save_fileName = save_path  + save_folder + "/" + "aug2" + name
        librosa.output.write_wav(save_fileName , augment_data, sampleRate)

        #change pitch a bit
        pitch_factor = np.random.normal(0,2)/100
        augment_data = pitch_shift(x,sampleRate,pitch_factor)
        save_fileName = save_path  + save_folder + "/" + "aug3" + name
        librosa.output.write_wav(save_fileName, augment_data, sampleRate)

        # speed up song
        augment_data = speed_change(x,1.1,sampleRate)
        save_fileName = save_path  + save_folder + "/" + "aug4" + name
        librosa.output.write_wav(save_fileName, augment_data, sampleRate)
        
        # slow down song
        augment_data = speed_change(x,0.9,sampleRate)
        save_fileName = save_path  + save_folder + "/" + "aug5" + name
        librosa.output.write_wav(save_fileName, augment_data, sampleRate)


        count = count + 1
        if(count % 10 == 0):
            print("Completed ",count," songs Augmentation")
print("Done...")
